In [21]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

In [3]:
data1 = np.random.rand(10000)
data2 = np.random.rand(10000)

#example dataframe with same structure, index can be viewed as time here
df = pd.DataFrame({'Temp': data1, 'Load': data2})
df

Temp      Load
0     0.547776  0.886892
1     0.695853  0.164058
2     0.383835  0.267596
3     0.077193  0.773319
4     0.197426  0.494024
...        ...       ...
9995  0.343432  0.054792
9996  0.434470  0.363905
9997  0.337740  0.060005
9998  0.028105  0.354644
9999  0.557684  0.738045

[10000 rows x 2 columns]

In [19]:
#ok lets say I have 10,000 data points. I want to split training testing into ~80/20
train_df = df * 0.8
test_df = df[-int(len(df)*0.2):]
train_df.shape, test_df.shape

((10000, 2), (2000, 2))

In [41]:
def prepare_data_nn(df, look_back, model='nn',  pred_type='N/A', train=0.80, val=0.10, test=0.10):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np) - look_back):
        row = [r for r in df_as_np[i:i+look_back]]
        X.append(row)
        y_values = df_as_np[i+look_back][0]
        y.append(y_values)
               
    X = np.array(X)
    y = np.array(y)
    
    length = len(X)
    
    
    x_train , y_train = X[:int((train*length))], y[:int((train*length))]
    x_val, y_val = X[int((train*length)):int((train+val)*length)], y[int((train*length)):int((train+val)*length)]
    x_test, y_test = X[-int(test*length):], y[-int(test*length):]
    
    
    
    return x_train, y_train, x_val, y_val, x_test, y_test
    

In [1]:
def check_missing_hourly_intervals(df):
    # Convert index to datetime if not already
    df.index = pd.to_datetime(df.index, utc=True)

    # Create a complete hourly time index
    full_time_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='H', tz='UTC')
    df_reindexed = df.reindex(full_time_index)

    # Identify missing timestamps
    missing_timestamps = df_reindexed[df_reindexed.isna().any(axis=1)]

    # Count missing data per day
    daily_missing = missing_timestamps.resample('D').size()

    return daily_missing.sort_values(ascending=False)

In [42]:
x1,y1 = prepare_data_nn(df,look_back=24)
x1.shape, y1.shape

((9976, 24, 2), (9976,))

In [51]:
x1[-int(.1*len(x1)):].shape

(997, 24, 2)

In [48]:
.8 * 9976

7980.8

In [34]:
df_as_np = df.to_numpy()
df_as_np[0][:]

array([0.54777552, 0.88689184])

In [ ]:
def rolling_forecast(df: pd.DataFrame, train_len: int, horizon: int, window: int, method: str):
    total_len = train_len + horizon
    end_idx = train_len
    
    if method == 'mean':
        pred_mean = []
        
        for i in range(train_len, total_len, window):
            mean = np.mean(df[:i].values)
            pred_mean.extend(mean for _ in range(window))
        
        return pred_mean
    
    if method == 'last':
        pred_last = []
        
        for i in range(train_len, total_len, window):
            last = df[:i].iloc[-1].values[0]
            pred_last.extend(last for _ in range(window))
            
        return pred_last
    
    if method == 'ARMA':
        pred_ARMA = []
        
        for i in range(train_len, total_len, window):
            model = SARIMAX(df[:i], order = (2,0,2))
            res = model.fit(disp=False)
            predictions = res.get_prediction(0, i + window -1)
            oos_pred = predictions.predicted_mean.iloc[-window:]
            pred_ARMA.extend(oos_pred)
            
        return pred_ARMA

In [ ]:
# train on 2 years and maintain a rolling forecast for another year

In [ ]:
def 